<a href="https://colab.research.google.com/github/brainer3220/Jeju_CreditCard_Compete/blob/master/%EC%A0%9C%EC%A3%BC_%EB%B9%85%EB%8D%B0%EC%9D%B4%ED%84%B0_%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from datetime import datetime
 
import plotly.graph_objs as go
import plotly.offline as py
import plotly.express as px
 
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, add_changepoints_to_plot
 
import seaborn as sns
import matplotlib.pyplot as plt

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Google Drive Sync

In [ ]:
Data = 'drive/My Drive/Brainer 공유 Drive/Colab/제주 빅데이터 경진대회/Data/'

In [ ]:
Submission = pd.read_csv(Data + "submission.csv")
Submission.head(3)

,id,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,0,202004,강원,건강보조식품 소매업,0
1,1,202004,강원,골프장 운영업,0
2,2,202004,강원,과실 및 채소 소매업,0


In [ ]:
df = pd.read_csv(Data + "201901-202003.csv")
df.head(3)

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
0,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,4,311200,4
1,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,7,1374500,8
2,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,6,818700,6


# Data Preprocessing

## SEX One Hot Encoding


In [ ]:
enc = OneHotEncoder()

SEX_List = np.array(df.SEX_CTGO_CD.tolist()).reshape(-1, 1)

enc.fit(SEX_List)
SEX_OneHot = enc.transform(SEX_List).toarray()
SEX_OneHot_Series = pd.DataFrame(SEX_OneHot).rename(columns = {0: 'Male', 1: 'FeMale'})

df = pd.concat([df, SEX_OneHot_Series], axis=1)

del SEX_OneHot, SEX_OneHot_Series, SEX_List, df['SEX_CTGO_CD']

## REG_YYMM(Integer) to TimeStamp

In [ ]:
REG_Date = pd.to_datetime(df['REG_YYMM'], format='%Y%m', errors='ignore')

del df['REG_YYMM']

df = pd.concat([REG_Date, df], axis=1)

del REG_Date

# Data Check

## Pearson Corrilation

In [ ]:
df.corr('pearson').drop('FLC').drop('Male').drop('FeMale')

,FLC,CSTMR_CNT,AMT,CNT,Male,FeMale
CSTMR_CNT,0.006774,1.000000,0.866339,0.930356,-0.008258,0.008258
AMT,0.041099,0.866339,1.000000,0.778102,0.005424,-0.005424
CNT,-0.001011,0.930356,0.778102,1.000000,-0.000469,0.000469


In [ ]:
df.head(3)

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,FLC,CSTMR_CNT,AMT,CNT,Male,FeMale
0,2019-01-01,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,4,311200,4,1.0,0.0
1,2019-01-01,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,7,1374500,8,1.0,0.0
2,2019-01-01,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,6,818700,6,0.0,1.0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24697792 entries, 0 to 24697791
Data columns (total 13 columns):
 #   Column        Dtype         
---  ------        -----         
 0   REG_YYMM      datetime64[ns]
 1   CARD_SIDO_NM  object        
 2   CARD_CCG_NM   object        
 3   STD_CLSS_NM   object        
 4   HOM_SIDO_NM   object        
 5   HOM_CCG_NM    object        
 6   AGE           object        
 7   FLC           int64         
 8   CSTMR_CNT     int64         
 9   AMT           int64         
 10  CNT           int64         
 11  Male          float64       
 12  FeMale        float64       
dtypes: datetime64[ns](1), float64(2), int64(4), object(6)
memory usage: 2.4+ GB


In [ ]:
df.describe()

,FLC,CSTMR_CNT,AMT,CNT,Male,FeMale
count,2.469779e+07,2.469779e+07,2.469779e+07,2.469779e+07,2.469779e+07,2.469779e+07
mean,2.804429e+00,6.196855e+01,2.441353e+06,1.260450e+02,5.873553e-01,4.126447e-01
std,1.375455e+00,3.559175e+02,1.726096e+07,9.669387e+02,4.923099e-01,4.923099e-01
min,1.000000e+00,3.000000e+00,2.000000e+01,1.000000e+00,0.000000e+00,0.000000e+00
25%,2.000000e+00,4.000000e+00,9.020000e+04,5.000000e+00,0.000000e+00,0.000000e+00
50%,3.000000e+00,8.000000e+00,2.526800e+05,1.200000e+01,1.000000e+00,0.000000e+00
75%,4.000000e+00,2.400000e+01,8.433372e+05,4.000000e+01,1.000000e+00,1.000000e+00
max,5.000000e+00,3.281300e+04,2.349524e+09,1.203570e+05,1.000000e+00,1.000000e+00


## Null Check

In [ ]:
df.isnull().sum()

REG_YYMM             0
CARD_SIDO_NM         0
CARD_CCG_NM      87213
STD_CLSS_NM          0
HOM_SIDO_NM          0
HOM_CCG_NM      147787
AGE                  0
FLC                  0
CSTMR_CNT            0
AMT                  0
CNT                  0
Male                 0
FeMale               0
dtype: int64

In [ ]:
for col in df.columns:
    msg = 'column: {:>10}\t Percent of NaN value: {:.2f}%'.format(col, 100 * (df[col].isnull().sum() / df[col].shape[0]))
    print(msg)

column:   REG_YYMM	 Percent of NaN value: 0.00%
column: CARD_SIDO_NM	 Percent of NaN value: 0.00%
column: CARD_CCG_NM	 Percent of NaN value: 0.35%
column: STD_CLSS_NM	 Percent of NaN value: 0.00%
column: HOM_SIDO_NM	 Percent of NaN value: 0.00%
column: HOM_CCG_NM	 Percent of NaN value: 0.60%
column:        AGE	 Percent of NaN value: 0.00%
column:        FLC	 Percent of NaN value: 0.00%
column:  CSTMR_CNT	 Percent of NaN value: 0.00%
column:        AMT	 Percent of NaN value: 0.00%
column:        CNT	 Percent of NaN value: 0.00%
column:       Male	 Percent of NaN value: 0.00%
column:     FeMale	 Percent of NaN value: 0.00%


# Graph Figure

In [ ]:
AmountDate = df.groupby('REG_YYMM').sum()

del AmountDate['FLC'], AmountDate['CSTMR_CNT'], AmountDate['CNT'], AmountDate['Male'], AmountDate['FeMale']

REG_YYMM =  df['REG_YYMM'].drop_duplicates()

if len(REG_YYMM) == len(AmountDate):
    AmountDate = pd.concat([pd.DataFrame(REG_YYMM.to_list(), columns =['REG_YYMM']) , pd.DataFrame(AmountDate['AMT'].to_list(), columns =['AMT'])], axis=1)

AmountDate.head(3)

,REG_YYMM,AMT
0,2019-01-01,4052317259025
1,2019-02-01,3634345832156
2,2019-03-01,4105409759731


## Monthly

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=AmountDate.REG_YYMM,
        y=AmountDate.AMT,
        name='날짜별 AMT'
    )
)

fig

## 달별, 도시별 카드 사용량

In [ ]:
SidoByDate = df.groupby(['REG_YYMM', 'CARD_SIDO_NM']).sum()
del SidoByDate['FLC'], SidoByDate['CSTMR_CNT'], SidoByDate['CNT'], SidoByDate['Male'], SidoByDate['FeMale']
pd.DataFrame(SidoByDate)

SidoByDate.reset_index(level=0, inplace=True)
SidoByDate.reset_index(level=0, inplace=True)

SidoByDate

,CARD_SIDO_NM,REG_YYMM,AMT
0,강원,2019-01-01,106565971761
1,경기,2019-01-01,926766490511
2,경남,2019-01-01,233964871967
3,경북,2019-01-01,178450257323
4,광주,2019-01-01,114443191321
...,...,...,...
250,전남,2020-03-01,116584279973
251,전북,2020-03-01,118942183199
252,제주,2020-03-01,71061095855
253,충남,2020-03-01,124325365579


In [ ]:
fig = px.line_3d(SidoByDate, x="REG_YYMM", y="CARD_SIDO_NM", z="AMT")
fig.show()

## 달별, 업종별 카드 사용량

In [ ]:
ClassByDate = df.groupby(['REG_YYMM', 'STD_CLSS_NM']).sum()
del ClassByDate['FLC'], ClassByDate['CSTMR_CNT'], ClassByDate['CNT'], ClassByDate['Male'], ClassByDate['FeMale']
pd.DataFrame(ClassByDate)

ClassByDate.reset_index(level=0, inplace=True)
ClassByDate.reset_index(level=0, inplace=True)

ClassByDate

,STD_CLSS_NM,REG_YYMM,AMT
0,건강보조식품 소매업,2019-01-01,8605074944
1,골프장 운영업,2019-01-01,11968748603
2,과실 및 채소 소매업,2019-01-01,44453112689
3,관광 민예품 및 선물용품 소매업,2019-01-01,955750428
4,그외 기타 분류안된 오락관련 서비스업,2019-01-01,1017500
...,...,...,...
610,피자 햄버거 샌드위치 및 유사 음식점업,2020-03-01,65972238656
611,한식 음식점업,2020-03-01,666573459086
612,호텔업,2020-03-01,5073633041
613,화장품 및 방향제 소매업,2020-03-01,31002789304


In [ ]:
fig = px.line_3d(ClassByDate, x="REG_YYMM", y="STD_CLSS_NM", z="AMT")
fig.show()

## 달별, 지역별, 업종별 카드 사용량

In [ ]:
SidoToClassByDate = df.groupby(['REG_YYMM', 'CARD_SIDO_NM','STD_CLSS_NM']).sum()
del SidoToClassByDate['FLC'], SidoToClassByDate['CSTMR_CNT'], SidoToClassByDate['CNT'], SidoToClassByDate['Male'], SidoToClassByDate['FeMale']
pd.DataFrame(SidoToClassByDate)

SidoToClassByDate.reset_index(level=0, inplace=True)
SidoToClassByDate.reset_index(level=0, inplace=True)
SidoToClassByDate.reset_index(level=0, inplace=True)

SidoToClassByDate

,STD_CLSS_NM,CARD_SIDO_NM,REG_YYMM,AMT
0,건강보조식품 소매업,강원,2019-01-01,148354746
1,골프장 운영업,강원,2019-01-01,1229200312
2,과실 및 채소 소매업,강원,2019-01-01,1286979106
3,관광 민예품 및 선물용품 소매업,강원,2019-01-01,30656770
4,그외 기타 스포츠시설 운영업,강원,2019-01-01,13101500
...,...,...,...,...
9433,피자 햄버거 샌드위치 및 유사 음식점업,충북,2020-03-01,1315245299
9434,한식 음식점업,충북,2020-03-01,16152482704
9435,호텔업,충북,2020-03-01,15248550
9436,화장품 및 방향제 소매업,충북,2020-03-01,428881434


## 성별별, 날짜별

In [ ]:
SexByDate = df.groupby(['REG_YYMM', 'Male']).sum()
del SexByDate['FLC'], SexByDate['CSTMR_CNT'], SexByDate['CNT'], SexByDate['FeMale']
pd.DataFrame(SexByDate)

SexByDate.reset_index(level=0, inplace=True)
SexByDate.reset_index(level=0, inplace=True)
SexByDate.reset_index(level=0, inplace=True)

SexByDate.head(3)

,index,Male,REG_YYMM,AMT
0,0,0.0,2019-01-01,1627661194843
1,1,1.0,2019-01-01,2424656064182
2,2,0.0,2019-02-01,1452925892309


In [ ]:
fig = px.scatter_3d(SexByDate, x='Male', y='REG_YYMM', z='AMT',
              color='Male')
fig.show()

## 업종별 성별

In [ ]:
ClassBySex = df.groupby(['STD_CLSS_NM', 'Male']).sum()
del ClassBySex['FLC'], ClassBySex['CSTMR_CNT'], ClassBySex['CNT'], ClassBySex['FeMale']
pd.DataFrame(SexByDate)

ClassBySex.reset_index(level=0, inplace=True)
ClassBySex.reset_index(level=0, inplace=True)
ClassBySex.reset_index(level=0, inplace=True)

ClassBySex.head(3)

,index,Male,STD_CLSS_NM,AMT
0,0,0.0,건강보조식품 소매업,60543938037
1,1,1.0,건강보조식품 소매업,44582426343
2,2,0.0,골프장 운영업,99447245024


In [ ]:
fig = px.scatter_3d(ClassBySex, x='Male', y='STD_CLSS_NM', z='AMT',
              color='Male')
fig.show()

# Learning and Training

In [ ]:
y_train = SidoToClassByDate.T[3:].T
y_train

,AMT
0,148354746
1,1229200312
2,1286979106
3,30656770
4,13101500
...,...
9433,1315245299
9434,16152482704
9435,15248550
9436,428881434


In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(SidoToClassByDate.T[2:-1].T)

X_trainDateEncoded = encoder.transform(SidoToClassByDate.T[2:-1].T)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



In [ ]:
SidoToClassByDate.T[:-2].T

,STD_CLSS_NM,CARD_SIDO_NM
0,건강보조식품 소매업,강원
1,골프장 운영업,강원
2,과실 및 채소 소매업,강원
3,관광 민예품 및 선물용품 소매업,강원
4,그외 기타 스포츠시설 운영업,강원
...,...,...
9433,피자 햄버거 샌드위치 및 유사 음식점업,충북
9434,한식 음식점업,충북
9435,호텔업,충북
9436,화장품 및 방향제 소매업,충북


In [ ]:
# list(Submission['REG_YYMM'].replace(202004, 15).replace(202007, 18))

In [ ]:
# del Submission['REG_YYMM']

Submit = pd.concat([Submission['STD_CLSS_NM'], Submission['CARD_SIDO_NM']], axis=1)
Submit = pd.get_dummies(Submit)
# del SubPlusTrain['AMT']
Submit

# Submission['STD_CLSS_NM'], Submission['CARD_SIDO_NM']
# SidoToClassByDate

,STD_CLSS_NM_건강보조식품 소매업,STD_CLSS_NM_골프장 운영업,STD_CLSS_NM_과실 및 채소 소매업,STD_CLSS_NM_관광 민예품 및 선물용품 소매업,STD_CLSS_NM_그외 기타 분류안된 오락관련 서비스업,STD_CLSS_NM_그외 기타 스포츠시설 운영업,STD_CLSS_NM_그외 기타 종합 소매업,STD_CLSS_NM_기타 대형 종합 소매업,STD_CLSS_NM_기타 수상오락 서비스업,STD_CLSS_NM_기타 외국식 음식점업,STD_CLSS_NM_기타 주점업,STD_CLSS_NM_기타음식료품위주종합소매업,STD_CLSS_NM_내항 여객 운송업,STD_CLSS_NM_마사지업,STD_CLSS_NM_면세점,STD_CLSS_NM_버스 운송업,STD_CLSS_NM_비알콜 음료점업,STD_CLSS_NM_빵 및 과자류 소매업,STD_CLSS_NM_서양식 음식점업,STD_CLSS_NM_수산물 소매업,STD_CLSS_NM_슈퍼마켓,STD_CLSS_NM_스포츠 및 레크레이션 용품 임대업,STD_CLSS_NM_여관업,STD_CLSS_NM_여행사업,STD_CLSS_NM_욕탕업,STD_CLSS_NM_육류 소매업,STD_CLSS_NM_일반유흥 주점업,STD_CLSS_NM_일식 음식점업,STD_CLSS_NM_자동차 임대업,STD_CLSS_NM_전시 및 행사 대행업,STD_CLSS_NM_정기 항공 운송업,STD_CLSS_NM_중식 음식점업,STD_CLSS_NM_차량용 가스 충전업,STD_CLSS_NM_차량용 주유소 운영업,STD_CLSS_NM_체인화 편의점,STD_CLSS_NM_택시 운송업,STD_CLSS_NM_피자 햄버거 샌드위치 및 유사 음식점업,STD_CLSS_NM_한식 음식점업,STD_CLSS_NM_호텔업,STD_CLSS_NM_화장품 및 방향제 소매업,STD_CLSS_NM_휴양콘도 운영업,CARD_SIDO_NM_강원,CARD_SIDO_NM_경기,CARD_SIDO_NM_경남,CARD_SIDO_NM_경북,CARD_SIDO_NM_광주,CARD_SIDO_NM_대구,CARD_SIDO_NM_대전,CARD_SIDO_NM_부산,CARD_SIDO_NM_서울,CARD_SIDO_NM_세종,CARD_SIDO_NM_울산,CARD_SIDO_NM_인천,CARD_SIDO_NM_전남,CARD_SIDO_NM_전북,CARD_SIDO_NM_제주,CARD_SIDO_NM_충남,CARD_SIDO_NM_충북
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1389,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1390,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1391,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1392,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [ ]:
x_train = pd.get_dummies(SidoToClassByDate.T[:-2].T)

x_train = pd.concat([pd.DataFrame(X_trainDateEncoded), x_train], axis=1)
x_train

,0,STD_CLSS_NM_건강보조식품 소매업,STD_CLSS_NM_골프장 운영업,STD_CLSS_NM_과실 및 채소 소매업,STD_CLSS_NM_관광 민예품 및 선물용품 소매업,STD_CLSS_NM_그외 기타 분류안된 오락관련 서비스업,STD_CLSS_NM_그외 기타 스포츠시설 운영업,STD_CLSS_NM_그외 기타 종합 소매업,STD_CLSS_NM_기타 대형 종합 소매업,STD_CLSS_NM_기타 수상오락 서비스업,STD_CLSS_NM_기타 외국식 음식점업,STD_CLSS_NM_기타 주점업,STD_CLSS_NM_기타음식료품위주종합소매업,STD_CLSS_NM_내항 여객 운송업,STD_CLSS_NM_마사지업,STD_CLSS_NM_면세점,STD_CLSS_NM_버스 운송업,STD_CLSS_NM_비알콜 음료점업,STD_CLSS_NM_빵 및 과자류 소매업,STD_CLSS_NM_서양식 음식점업,STD_CLSS_NM_수산물 소매업,STD_CLSS_NM_슈퍼마켓,STD_CLSS_NM_스포츠 및 레크레이션 용품 임대업,STD_CLSS_NM_여관업,STD_CLSS_NM_여행사업,STD_CLSS_NM_욕탕업,STD_CLSS_NM_육류 소매업,STD_CLSS_NM_일반유흥 주점업,STD_CLSS_NM_일식 음식점업,STD_CLSS_NM_자동차 임대업,STD_CLSS_NM_전시 및 행사 대행업,STD_CLSS_NM_정기 항공 운송업,STD_CLSS_NM_중식 음식점업,STD_CLSS_NM_차량용 가스 충전업,STD_CLSS_NM_차량용 주유소 운영업,STD_CLSS_NM_체인화 편의점,STD_CLSS_NM_택시 운송업,STD_CLSS_NM_피자 햄버거 샌드위치 및 유사 음식점업,STD_CLSS_NM_한식 음식점업,STD_CLSS_NM_호텔업,STD_CLSS_NM_화장품 및 방향제 소매업,STD_CLSS_NM_휴양콘도 운영업,CARD_SIDO_NM_강원,CARD_SIDO_NM_경기,CARD_SIDO_NM_경남,CARD_SIDO_NM_경북,CARD_SIDO_NM_광주,CARD_SIDO_NM_대구,CARD_SIDO_NM_대전,CARD_SIDO_NM_부산,CARD_SIDO_NM_서울,CARD_SIDO_NM_세종,CARD_SIDO_NM_울산,CARD_SIDO_NM_인천,CARD_SIDO_NM_전남,CARD_SIDO_NM_전북,CARD_SIDO_NM_제주,CARD_SIDO_NM_충남,CARD_SIDO_NM_충북
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9433,14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
9434,14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
9435,14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
9436,14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [ ]:
y_train[0:10]

,AMT
0,148354746
1,1229200312
2,1286979106
3,30656770
4,13101500
5,533330540
6,8442324759
7,4224000
8,1164632486
9,326489689


In [ ]:
Submit = pd.concat([pd.DataFrame(Submission['REG_YYMM'].replace(202004, 15).replace(202007, 18)), Submit], axis=1)
Submit

,REG_YYMM,STD_CLSS_NM_건강보조식품 소매업,STD_CLSS_NM_골프장 운영업,STD_CLSS_NM_과실 및 채소 소매업,STD_CLSS_NM_관광 민예품 및 선물용품 소매업,STD_CLSS_NM_그외 기타 분류안된 오락관련 서비스업,STD_CLSS_NM_그외 기타 스포츠시설 운영업,STD_CLSS_NM_그외 기타 종합 소매업,STD_CLSS_NM_기타 대형 종합 소매업,STD_CLSS_NM_기타 수상오락 서비스업,STD_CLSS_NM_기타 외국식 음식점업,STD_CLSS_NM_기타 주점업,STD_CLSS_NM_기타음식료품위주종합소매업,STD_CLSS_NM_내항 여객 운송업,STD_CLSS_NM_마사지업,STD_CLSS_NM_면세점,STD_CLSS_NM_버스 운송업,STD_CLSS_NM_비알콜 음료점업,STD_CLSS_NM_빵 및 과자류 소매업,STD_CLSS_NM_서양식 음식점업,STD_CLSS_NM_수산물 소매업,STD_CLSS_NM_슈퍼마켓,STD_CLSS_NM_스포츠 및 레크레이션 용품 임대업,STD_CLSS_NM_여관업,STD_CLSS_NM_여행사업,STD_CLSS_NM_욕탕업,STD_CLSS_NM_육류 소매업,STD_CLSS_NM_일반유흥 주점업,STD_CLSS_NM_일식 음식점업,STD_CLSS_NM_자동차 임대업,STD_CLSS_NM_전시 및 행사 대행업,STD_CLSS_NM_정기 항공 운송업,STD_CLSS_NM_중식 음식점업,STD_CLSS_NM_차량용 가스 충전업,STD_CLSS_NM_차량용 주유소 운영업,STD_CLSS_NM_체인화 편의점,STD_CLSS_NM_택시 운송업,STD_CLSS_NM_피자 햄버거 샌드위치 및 유사 음식점업,STD_CLSS_NM_한식 음식점업,STD_CLSS_NM_호텔업,STD_CLSS_NM_화장품 및 방향제 소매업,STD_CLSS_NM_휴양콘도 운영업,CARD_SIDO_NM_강원,CARD_SIDO_NM_경기,CARD_SIDO_NM_경남,CARD_SIDO_NM_경북,CARD_SIDO_NM_광주,CARD_SIDO_NM_대구,CARD_SIDO_NM_대전,CARD_SIDO_NM_부산,CARD_SIDO_NM_서울,CARD_SIDO_NM_세종,CARD_SIDO_NM_울산,CARD_SIDO_NM_인천,CARD_SIDO_NM_전남,CARD_SIDO_NM_전북,CARD_SIDO_NM_제주,CARD_SIDO_NM_충남,CARD_SIDO_NM_충북
0,15,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,15,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,15,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,15,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,15,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1389,18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1390,18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1391,18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1392,18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


### LinearRegression

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso

mlr = LinearRegression()
mlr.fit(x_train, y_train)

LinearPredict =  mlr.predict(Submit)

Submission['AMT'] = LinearPredict
LinearPredict = Submission
LinearPredict

,id,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,0,202004,강원,건강보조식품 소매업,-3.061383e+09
1,1,202004,강원,골프장 운영업,-1.137574e+09
2,2,202004,강원,과실 및 채소 소매업,-1.046086e+09
3,3,202004,강원,관광 민예품 및 선물용품 소매업,-3.445621e+09
4,4,202004,강원,그외 기타 분류안된 오락관련 서비스업,-3.063611e+09
...,...,...,...,...,...
1389,1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,3.416392e+08
1390,1390,202007,충북,한식 음식점업,4.989787e+10
1391,1391,202007,충북,호텔업,-3.496083e+09
1392,1392,202007,충북,화장품 및 방향제 소매업,-1.466434e+09


In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        # x=pd.to_datetime(Submission['REG_YYMM'], format='%Y%m', errors='ignore'),
        y=LinearPredict.AMT,
        name='최종 제출'
    )
)

fig

fig = px.scatter_3d(LinearPredict, x=pd.to_datetime(LinearPredict['REG_YYMM'], format='%Y%m', errors='ignore'), y='CARD_SIDO_NM', z='AMT',
              color='STD_CLSS_NM')
fig.show()

In [ ]:
Submission = Submission.set_index('id')
Submission.to_csv('drive/My Drive/Brainer 공유 Drive/Colab/제주 빅데이터 경진대회/LinearPredict.csv', encoding='utf-8-sig')

### Ridge Regression

In [ ]:
ridge_reg = Ridge()
ridge_reg.fit(x_train, y_train)

RidgePredict = ridge_reg.predict(Submit)

Submission['AMT'] = RidgePredict
RidgePredict = Submission
RidgePredict

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,-3.036541e+09
1,202004,강원,골프장 운영업,-1.120230e+09
2,202004,강원,과실 및 채소 소매업,-1.029500e+09
3,202004,강원,관광 민예품 및 선물용품 소매업,-3.416798e+09
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,-2.961344e+09
...,...,...,...,...
1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,3.546327e+08
1390,202007,충북,한식 음식점업,4.971756e+10
1391,202007,충북,호텔업,-3.465037e+09


In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        # x=pd.to_datetime(Submission['REG_YYMM'], format='%Y%m', errors='ignore'),
        y=RidgePredict.AMT,
        name='최종 제출'
    )
)

fig

fig = px.scatter_3d(RidgePredict, x=pd.to_datetime(RidgePredict['REG_YYMM'], format='%Y%m', errors='ignore'), y='CARD_SIDO_NM', z='AMT',
              color='STD_CLSS_NM')
fig.show()

In [ ]:
# Submission = Submission.set_index('id')
Submission.to_csv('drive/My Drive/Brainer 공유 Drive/Colab/제주 빅데이터 경진대회/RidgePredict.csv', encoding='utf-8-sig')

### Lasso Regression

In [ ]:
lasso_reg = Lasso()
lasso_reg.fit(x_train, y_train)

LassoPredict = lasso_reg.predict(Submit)

LassoPredict = Submission['AMT'] = LassoPredict

LassoPredict = Submission
LassoPredict

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.9184752210119868e+23, tolerance: 3.398124001385555e+20



,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,-3.062061e+09
1,202004,강원,골프장 운영업,-1.138235e+09
2,202004,강원,과실 및 채소 소매업,-1.047150e+09
3,202004,강원,관광 민예품 및 선물용품 소매업,-3.443992e+09
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,-3.063013e+09
...,...,...,...,...
1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,3.418329e+08
1390,202007,충북,한식 음식점업,4.989834e+10
1391,202007,충북,호텔업,-3.495694e+09


In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        # x=pd.to_datetime(Submission['REG_YYMM'], format='%Y%m', errors='ignore'),
        y=LassoPredict.AMT,
        name='최종 제출'
    )
)

fig

fig = px.scatter_3d(LassoPredict, x=pd.to_datetime(LassoPredict['REG_YYMM'], format='%Y%m', errors='ignore'), y='CARD_SIDO_NM', z='AMT',
              color='STD_CLSS_NM')
fig.show()

In [ ]:
# Submission = Submission.set_index('id')
Submission.to_csv('drive/My Drive/Brainer 공유 Drive/Colab/제주 빅데이터 경진대회/LassoPredict.csv', encoding='utf-8-sig')

### RandomForestRegressor

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

k_fold = KFold(n_splits=10, shuffle=True, random_state=0)

max_depth_list = []

model = RandomForestRegressor(n_estimators=100,
                              n_jobs=-1,
                              random_state=0)
model

model.fit(x_train, y_train)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=-1, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

In [ ]:
# 예측
predictions = model.predict(Submit)

print(predictions.shape)
predictions[0:10]

(1394,)


array([9.41784526e+07, 2.35038113e+09, 9.88906754e+08, 1.66280007e+07,
       9.60005200e+05, 2.78505700e+06, 4.73531727e+08, 6.59972454e+09,
       9.51445000e+05, 8.58655018e+08])

In [ ]:
Submission['AMT'] = predictions
Submission.head(3)

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,9.417845e+07
1,202004,강원,골프장 운영업,2.350381e+09
2,202004,강원,과실 및 채소 소매업,9.889068e+08


In [ ]:
# Submission = Submission.set_index('id')
Submission.to_csv('drive/My Drive/Brainer 공유 Drive/Colab/제주 빅데이터 경진대회/Submission.csv', encoding='utf-8-sig')

In [ ]:
Submission.head(3)

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,9.417845e+07
1,202004,강원,골프장 운영업,2.350381e+09
2,202004,강원,과실 및 채소 소매업,9.889068e+08


In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        # x=pd.to_datetime(Submission['REG_YYMM'], format='%Y%m', errors='ignore'),
        y=Submission.AMT,
        name='최종 제출'
    )
)

fig

fig = px.scatter_3d(Submission, x=pd.to_datetime(Submission['REG_YYMM'], format='%Y%m', errors='ignore'), y='CARD_SIDO_NM', z='AMT',
              color='STD_CLSS_NM')
fig.show()

# 카드 이용건수와, 이용 고객수 분석

## 카드 이용 고객수 분석과 예측

In [66]:
tmp = df.groupby(['REG_YYMM', 'CARD_SIDO_NM','STD_CLSS_NM']).sum()
tmp.reset_index(level=2, inplace=True)
tmp.reset_index(level=1, inplace=True)
tmp.reset_index(level=0, inplace=True)
 
del tmp['REG_YYMM'], tmp['CARD_SIDO_NM'], tmp['STD_CLSS_NM'], tmp['FLC'], tmp['AMT'], 
tmp = pd.concat([SidoToClassByDate, tmp], axis=1)
 
tmp = tmp[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'CNT', 'CSTMR_CNT', 'Male', 'FeMale', 'AMT']]
# tmp = SidoToClassByDate[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]

# tmp = tmp.reindex(columns=['REG_YYMM','STD_CLSS_NM'])
 
tmp.head(3)

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,CNT,CSTMR_CNT,Male,FeMale,AMT
0,2019-01-01,강원,건강보조식품 소매업,1128,1080,44.0,50.0,148354746
1,2019-01-01,강원,골프장 운영업,33379,18489,1317.0,815.0,1229200312
2,2019-01-01,강원,과실 및 채소 소매업,31050,23566,1140.0,1022.0,1286979106


### CSTMR_CNT 분석과 예측

In [89]:
X_CSTMR_CNT = pd.concat([pd.DataFrame(X_trainDateEncoded), tmp['CARD_SIDO_NM'], tmp['STD_CLSS_NM']], axis=1)
X_CSTMR_CNT = pd.get_dummies(X_CSTMR_CNT)
X_CSTMR_CNT.head(3)

,0,CARD_SIDO_NM_강원,CARD_SIDO_NM_경기,CARD_SIDO_NM_경남,CARD_SIDO_NM_경북,CARD_SIDO_NM_광주,CARD_SIDO_NM_대구,CARD_SIDO_NM_대전,CARD_SIDO_NM_부산,CARD_SIDO_NM_서울,CARD_SIDO_NM_세종,CARD_SIDO_NM_울산,CARD_SIDO_NM_인천,CARD_SIDO_NM_전남,CARD_SIDO_NM_전북,CARD_SIDO_NM_제주,CARD_SIDO_NM_충남,CARD_SIDO_NM_충북,STD_CLSS_NM_건강보조식품 소매업,STD_CLSS_NM_골프장 운영업,STD_CLSS_NM_과실 및 채소 소매업,STD_CLSS_NM_관광 민예품 및 선물용품 소매업,STD_CLSS_NM_그외 기타 분류안된 오락관련 서비스업,STD_CLSS_NM_그외 기타 스포츠시설 운영업,STD_CLSS_NM_그외 기타 종합 소매업,STD_CLSS_NM_기타 대형 종합 소매업,STD_CLSS_NM_기타 수상오락 서비스업,STD_CLSS_NM_기타 외국식 음식점업,STD_CLSS_NM_기타 주점업,STD_CLSS_NM_기타음식료품위주종합소매업,STD_CLSS_NM_내항 여객 운송업,STD_CLSS_NM_마사지업,STD_CLSS_NM_면세점,STD_CLSS_NM_버스 운송업,STD_CLSS_NM_비알콜 음료점업,STD_CLSS_NM_빵 및 과자류 소매업,STD_CLSS_NM_서양식 음식점업,STD_CLSS_NM_수산물 소매업,STD_CLSS_NM_슈퍼마켓,STD_CLSS_NM_스포츠 및 레크레이션 용품 임대업,STD_CLSS_NM_여관업,STD_CLSS_NM_여행사업,STD_CLSS_NM_욕탕업,STD_CLSS_NM_육류 소매업,STD_CLSS_NM_일반유흥 주점업,STD_CLSS_NM_일식 음식점업,STD_CLSS_NM_자동차 임대업,STD_CLSS_NM_전시 및 행사 대행업,STD_CLSS_NM_정기 항공 운송업,STD_CLSS_NM_중식 음식점업,STD_CLSS_NM_차량용 가스 충전업,STD_CLSS_NM_차량용 주유소 운영업,STD_CLSS_NM_체인화 편의점,STD_CLSS_NM_택시 운송업,STD_CLSS_NM_피자 햄버거 샌드위치 및 유사 음식점업,STD_CLSS_NM_한식 음식점업,STD_CLSS_NM_호텔업,STD_CLSS_NM_화장품 및 방향제 소매업,STD_CLSS_NM_휴양콘도 운영업
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [80]:
CSTMR_CNT = (tmp['CSTMR_CNT']).tolist()
# CSTMR_CNT

In [81]:
model.fit(X_CSTMR_CNT, CSTMR_CNT)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=-1, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

#### 예측 

In [82]:
CSTMR_CNT_Predict = model.predict(X_CSTMR_CNT)

print(CSTMR_CNT_Predict.shape)
CSTMR_CNT_Predict[0:10]

(9438,)


array([1.118170e+03, 1.819824e+04, 2.200757e+04, 1.003720e+03,
       1.033600e+02, 6.608720e+03, 1.099826e+05, 1.129200e+02,
       3.685328e+04, 2.236860e+03])

In [88]:
CSTMR_CNT[0:10]

[1080, 18489, 23566, 1059, 121, 6738, 111070, 110, 37759, 2304]